### Experimenting with catboost
In this notebook, we will attempt to get better performance using catboost after tuning with python and check its accuracy in comparison to other models.

In [15]:
import optuna
#from optuna.integration import CatBoostPruningCallback include after installing latest optuna that supports pruning callback for catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder


#Test and train without preprocessing of categorical variables
train_df = pd.read_csv("/home/mwanikii/Documents/data_science_projects/financial_inclusivity/input/Train.csv")
test_df = pd.read_csv("/home/mwanikii/Documents/data_science_projects/financial_inclusivity/input/Test.csv")


#Encoding bank_account column
LE = LabelEncoder()
train_df["bank_account"] = LE.fit_transform(train_df["bank_account"])

#X and Y definition
X_train = train_df.drop(["uniqueid", "year", "bank_account"], axis=1)
y_train = train_df["bank_account"]
X_test = test_df.drop(["uniqueid", "year"], axis=1)

#Train_test split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=10000, test_size=0.3 )


#Optuna trial logic
def objective(trial, X, y):
    parameter_grid = {
        "iterations": trial.suggest_categorical("iterations", [10000]),#alias n_estimators
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 0, 100, step=5),#alias lambda
        "depth": trial.suggest_int("depth", 3, 12) #alias max_depth
        #alias num_leaves "max_leaves": trial.suggest_int("max_leaves", 20, 3000, step=20)
    }
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)
    cv_scores = np.empty(5)

    cat_features = np.where(X.dtypes != np.int64)[0]

    for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = CatBoostClassifier(loss_function="Logloss", **parameter_grid)
        model.fit(X_train,
                  y_train,
                  eval_set=[(X_val, y_val)],
                  early_stopping_rounds=100,
                  cat_features=cat_features
                  ) #callbacks=[CatBoostPruningCallback(trial, "validation_0-logloss")] include in optuna that supports pruning callbacks for catboost

        y_preds = model.predict(X_val)
        cv_scores[idx] = log_loss(y_val, y_preds)


    return np.mean(cv_scores)

study = optuna.create_study(direction="minimize", study_name="CatBoostClassifier")
func = lambda trial: objective(trial, X_train, y_train)
study.optimize(func, n_trials = 20)

[I 2022-08-24 21:16:52,851] A new study created in memory with name: CatBoostClassifier


0:	learn: 0.4914830	test: 0.4936758	best: 0.4936758 (0)	total: 32.5ms	remaining: 5m 24s
1:	learn: 0.3975830	test: 0.3987888	best: 0.3987888 (1)	total: 49.8ms	remaining: 4m 8s
2:	learn: 0.3522874	test: 0.3552087	best: 0.3552087 (2)	total: 64.9ms	remaining: 3m 36s
3:	learn: 0.3271411	test: 0.3309949	best: 0.3309949 (3)	total: 80.7ms	remaining: 3m 21s
4:	learn: 0.3116003	test: 0.3180736	best: 0.3180736 (4)	total: 97.6ms	remaining: 3m 15s
5:	learn: 0.3010132	test: 0.3086878	best: 0.3086878 (5)	total: 108ms	remaining: 2m 59s
6:	learn: 0.2947382	test: 0.3028530	best: 0.3028530 (6)	total: 118ms	remaining: 2m 49s
7:	learn: 0.2891556	test: 0.2971696	best: 0.2971696 (7)	total: 131ms	remaining: 2m 43s
8:	learn: 0.2865909	test: 0.2949242	best: 0.2949242 (8)	total: 142ms	remaining: 2m 37s
9:	learn: 0.2848495	test: 0.2940185	best: 0.2940185 (9)	total: 150ms	remaining: 2m 29s
10:	learn: 0.2839980	test: 0.2929191	best: 0.2929191 (10)	total: 155ms	remaining: 2m 20s
11:	learn: 0.2829570	test: 0.2920651	

[I 2022-08-24 21:17:11,731] Trial 0 finished with value: 3.7421030559304675 and parameters: {'iterations': 10000, 'learning_rate': 0.2644999333830951, 'l2_leaf_reg': 55, 'depth': 7}. Best is trial 0 with value: 3.7421030559304675.


0:	learn: 0.5438400	test: 0.5457725	best: 0.5457725 (0)	total: 34.4ms	remaining: 5m 43s
1:	learn: 0.4616702	test: 0.4632052	best: 0.4632052 (1)	total: 51.4ms	remaining: 4m 16s
2:	learn: 0.4075701	test: 0.4100104	best: 0.4100104 (2)	total: 82.2ms	remaining: 4m 33s
3:	learn: 0.3656668	test: 0.3688212	best: 0.3688212 (3)	total: 115ms	remaining: 4m 48s
4:	learn: 0.3404423	test: 0.3451217	best: 0.3451217 (4)	total: 350ms	remaining: 11m 39s
5:	learn: 0.3283128	test: 0.3329198	best: 0.3329198 (5)	total: 357ms	remaining: 9m 55s
6:	learn: 0.3157242	test: 0.3207343	best: 0.3207343 (6)	total: 378ms	remaining: 8m 59s
7:	learn: 0.3045310	test: 0.3106331	best: 0.3106331 (7)	total: 612ms	remaining: 12m 43s
8:	learn: 0.2962149	test: 0.3036468	best: 0.3036468 (8)	total: 637ms	remaining: 11m 47s
9:	learn: 0.2902564	test: 0.2993733	best: 0.2993733 (9)	total: 868ms	remaining: 14m 27s
10:	learn: 0.2849253	test: 0.2963494	best: 0.2963494 (10)	total: 1.1s	remaining: 16m 40s
11:	learn: 0.2833131	test: 0.29509

[I 2022-08-24 21:19:28,023] Trial 1 finished with value: 3.7798555497248296 and parameters: {'iterations': 10000, 'learning_rate': 0.17346978119374515, 'l2_leaf_reg': 30, 'depth': 12}. Best is trial 0 with value: 3.7421030559304675.


...

[I 2022-08-24 21:30:18,897] Trial 17 finished with value: 3.727410032455706 and parameters: {'iterations': 10000, 'learning_rate': 0.19218203158428052, 'l2_leaf_reg': 90, 'depth': 7}. Best is trial 10 with value: 3.7211221519191975.


0:	learn: 0.5869246	test: 0.5869808	best: 0.5869808 (0)	total: 21.6ms	remaining: 3m 36s
1:	learn: 0.5151407	test: 0.5145585	best: 0.5145585 (1)	total: 42.8ms	remaining: 3m 33s
2:	learn: 0.4597600	test: 0.4586024	best: 0.4586024 (2)	total: 61ms	remaining: 3m 23s
3:	learn: 0.4231134	test: 0.4224039	best: 0.4224039 (3)	total: 84.7ms	remaining: 3m 31s
4:	learn: 0.3950433	test: 0.3947815	best: 0.3947815 (4)	total: 99.4ms	remaining: 3m 18s
5:	learn: 0.3727933	test: 0.3731186	best: 0.3731186 (5)	total: 108ms	remaining: 3m
6:	learn: 0.3565479	test: 0.3574844	best: 0.3574844 (6)	total: 116ms	remaining: 2m 45s
7:	learn: 0.3436332	test: 0.3449096	best: 0.3449096 (7)	total: 123ms	remaining: 2m 33s
8:	learn: 0.3340782	test: 0.3358271	best: 0.3358271 (8)	total: 132ms	remaining: 2m 26s
9:	learn: 0.3264043	test: 0.3286099	best: 0.3286099 (9)	total: 139ms	remaining: 2m 19s
10:	learn: 0.3187171	test: 0.3213699	best: 0.3213699 (10)	total: 148ms	remaining: 2m 14s
11:	learn: 0.3129061	test: 0.3158766	best:

[I 2022-08-24 21:30:43,286] Trial 18 finished with value: 3.737904653843336 and parameters: {'iterations': 10000, 'learning_rate': 0.1203891958731067, 'l2_leaf_reg': 65, 'depth': 5}. Best is trial 10 with value: 3.7211221519191975.


0:	learn: 0.6733349	test: 0.6736289	best: 0.6736289 (0)	total: 12ms	remaining: 2m
1:	learn: 0.6547470	test: 0.6552458	best: 0.6552458 (1)	total: 22ms	remaining: 1m 49s
2:	learn: 0.6372221	test: 0.6378618	best: 0.6378618 (2)	total: 29.1ms	remaining: 1m 37s
3:	learn: 0.6239144	test: 0.6245656	best: 0.6245656 (3)	total: 67.5ms	remaining: 2m 48s
4:	learn: 0.6116313	test: 0.6122317	best: 0.6122317 (4)	total: 74.1ms	remaining: 2m 28s
5:	learn: 0.5969045	test: 0.5975499	best: 0.5975499 (5)	total: 81.5ms	remaining: 2m 15s
6:	learn: 0.5856950	test: 0.5865489	best: 0.5865489 (6)	total: 88ms	remaining: 2m 5s
7:	learn: 0.5727781	test: 0.5733852	best: 0.5733852 (7)	total: 94.3ms	remaining: 1m 57s
8:	learn: 0.5609978	test: 0.5613492	best: 0.5613492 (8)	total: 99.8ms	remaining: 1m 50s
9:	learn: 0.5478080	test: 0.5481531	best: 0.5481531 (9)	total: 105ms	remaining: 1m 45s
10:	learn: 0.5361055	test: 0.5362895	best: 0.5362895 (10)	total: 110ms	remaining: 1m 39s
11:	learn: 0.5256614	test: 0.5257078	best: 

[I 2022-08-24 21:31:34,765] Trial 19 finished with value: 3.7358025410953695 and parameters: {'iterations': 10000, 'learning_rate': 0.02285546549489667, 'l2_leaf_reg': 45, 'depth': 3}. Best is trial 10 with value: 3.7211221519191975.


In [16]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")
    

	Best value (rmse): 3.72112
	Best params:
		iterations: 10000
		learning_rate: 0.2979761615880641
		l2_leaf_reg: 100
		depth: 3
